In [ ]:
from bs4 import BeautifulSoup
import requests
from nltk.stem import SnowballStemmer
import re
import os

In [ ]:
# Ve a https://whatmyuseragent.com/, copia y pega el resultado
headers = {
    'User-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0'
}
# Ocupamos pasarlo como parametro en las requests a la web para que no nos tome como un bot/script (pero osea si somos)

In [ ]:
"""Funcion que de webscrapping a la web del diccionario para
obtener los links hacia las definiciones de cada concepto"""
def get_definitions_links(url):
    data = requests.get(url, headers=headers).text # Obteniendo el html
    soup = BeautifulSoup(data,"lxml")  # parseamos el html en el objeto soup
    links = soup.find_all("a",{"class":"item-link"}) # Obtenemos todos los tags "a" (usado en html para links) presentes
    links = [link["href"] for link in links] # Almacenamos solo el atributo href (link)
    return links

In [ ]:
"""
Funcion para obtener todos los links de las definiciones que empiezen por una letra en específico.
Parametros:  letra (letra a scrappear en el diccionario), pag_inicial (numero de pagina inicial a scrappear),
pag_final (numero de pagina final)
"""
def find_all_links_in_letter(letra, pag_inicial, pag_final):
    all_links = []
    for i in range(pag_inicial, pag_final+1):
        url = "https://www.cun.es/diccionario-medico?letter={}&page={}".format(letra, i)
        links = get_definitions_links(url)
        all_links.extend(links)
    return all_links

In [ ]:
all_links = find_all_links_in_letter(letra='n', pag_inicial=10, pag_final=15)

In [ ]:
print(all_links)
print(len(all_links))

['https://www.cun.es/diccionario-medico/terminos/neuropatia-diabetica-sensitivomotora', 'https://www.cun.es/diccionario-medico/terminos/neuropatia-optica-isquemica', 'https://www.cun.es/diccionario-medico/terminos/neuropatia-periferica', 'https://www.cun.es/diccionario-medico/terminos/neuropatia-periferica-interes-quirurgico', 'https://www.cun.es/diccionario-medico/terminos/neuropeptido', 'https://www.cun.es/diccionario-medico/terminos/neuropeptido-y', 'https://www.cun.es/diccionario-medico/terminos/neuroporo', 'https://www.cun.es/diccionario-medico/terminos/neuropsicologia', 'https://www.cun.es/diccionario-medico/terminos/neurorradiologia', 'https://www.cun.es/diccionario-medico/terminos/neurorradiologico', 'https://www.cun.es/diccionario-medico/terminos/neurorradiologo', 'https://www.cun.es/diccionario-medico/terminos/neurorrafia', 'https://www.cun.es/diccionario-medico/terminos/neurorretinitis', 'https://www.cun.es/diccionario-medico/terminos/neurosecrecion', 'https://www.cun.es/dic

In [ ]:
# Declaramos nuestro stemmer (para obtener la raíz de una palabra):
stemmer = SnowballStemmer('spanish')

In [ ]:
# Directorio para guardar los archivos
#directorio = 'C:/Users/C-Ramirez/Documents/LCC/NLP/ProyectoFinal/'#defs'
directorio = 'G:/Mi unidad/PLN_UADY/ProyectoFinal/defs'

In [ ]:
# Funcion que elimina caracteres especiales de regex en palabras
def escape(texto):
    # Lista de caracteres especiales en expresiones regulares
    caracteres_especiales = ".^$*+?{}[]\\|()"

    # Reemplazar cada carácter especial con su versión escapada
    for caracter in caracteres_especiales:
        texto = texto.replace(caracter, "")
    return texto

In [ ]:
"""Funcion que hace webscrapping a las paginas con las definiciones de cada concepto, y las devuelve"""
def get_defs():
    for link in all_links:
        try:
            dt = requests.get(link, headers=headers).text
        except:
            print(f"Error solicitando datos de: {link}")
        tmp_soup = BeautifulSoup(dt,"html5lib")
        concept = tmp_soup.find("strong")
        concept = concept.text
        concept = re.sub(r'/.*', "", concept)
        concept = concept.lower()
        # Encontramos la etiqueta section en la que se encuentra la definicion
        section = tmp_soup.find("section", {"class":"textImageComponent textImageComponent"})
        # Encontramos todos los tags "p" ya que es el formato que usa la web para las definiciones
        paragraphs = section.find_all("p")
        # Unimos todos los textos en cada parrrafo y almacenamos
        complete_text = ""
        for p in paragraphs:
            s = p.text
            if(not s): continue
            if (s[0]=='¿' and s[-1]=='?') or s[0]=='©' or s[0]=='>': continue
            complete_text+=p.text
            complete_text+=" "

        complete_text = complete_text.strip()
        # Obtenemos la raíz del concepto y eliminamos apariciones, en caso de conceptos de mas de una palabra dividimos y hacemos por cada palabra
        texto_sin_concepto = complete_text.lower()
        for word in concept.split():
            stemmed_concept = stemmer.stem(word)
            # Reemplazamos las apariciones de las palabras que coincidan en su raíz con el concepto
            texto_sin_concepto = re.sub(r'(\b(?:la|los|el|las|lo|un|unos|de|al|una|del)\s?)?\b({}|{})\w*\b([.,]?)'.format(escape(stemmed_concept), escape(word)), '',texto_sin_concepto)

        texto_sin_concepto = re.sub(r'""', '', texto_sin_concepto) # Eliminamos comillas sobrantes
        texto_sin_concepto = re.sub(r'\s+', ' ', texto_sin_concepto) # Eliminamos espacios sobrantes
        # Escribimos el archivo
        if (texto_sin_concepto):
            path = os.path.join(directorio, "{}.txt".format(concept))
            with open(path, "w", encoding="utf-8") as file:
                try:
                    file.write(texto_sin_concepto.strip())
                except:
                    print(f"Error en escritura de archivo: {concept}, {link}")
        else:
            print(f"Error en escritura de archivo: {concept} , {link}")

In [ ]:
get_defs()

Error solicitando datos de: https://www.cun.es/diccionario-medico/terminos/neurotomia
Error solicitando datos de: https://www.cun.es/diccionario-medico/terminos/nilia
Error solicitando datos de: https://www.cun.es/diccionario-medico/terminos/nodulos-koeppe
